In [3]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv

In [15]:
class Skill:
    def __init__(self,name,dir, keyword , groups=None,prerequisites= None):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join(dir, filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = keyword  # keyword for searching LLM
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)
        self.prerequisite  = set()
        if prerequisites is not None:
            self.UpdatePrerequisites(prerequisites)
        

    def UpdateGroupSet(self,groups):
        self.group_set.update(groups)
        #print("skill group set updated.")
      
     
    def UpdatePrerequisites(self,prerequisites):
        self.prerequisite.update(prerequisites)
        #print("prerequisite updated.")
        
class Group:
    def __init__(self,name,skills):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("group", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " technology" # keyword for searching LLM
        self.skill_set = skills

    def UpdateSkillSet(self,skill):
        self.skill_set.update(skill)
        #print("group skill set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword

class Role:
    def __init__(self,name,skills,groups):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("role", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " role" # keyword for searching LLM
        self.skill_set = set()
        if skills is not None:    
            self.UpdateSkillSet(skills)
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)

    def UpdateSkillSet(self,skills):
        if skills is not None:    
            self.skill_set.update(skills)
            #print("group skill set updated.")

    def UpdateGroupSet(self,groups):
        if groups is not None:   
            self.group_set.update(groups)
            #print("skill group set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword
    
    
        

In [16]:
class TechStack:
    def __init__(self):
        self.skill_dict_list = {}
        self.group_dict_list = {}
        self.alias_dict_list = {}
        self.ignore_set = set()
        self.not_found_set = set()
        self.role_dict_list = {}

               
    def AddSkillDictList(self,name,path,keyword,groups=None,prerequisites = None):
        if name in self.alias_dict_list:
            name = self.alias_dict_list[name]
            
        if name not in self.skill_dict_list:
            self.skill_dict_list[name] = Skill(name,path,keyword,groups,prerequisites)
            #print(name,"added in skill_dict_list.")
            if groups is not None:
                for g in groups:
                    if g in self.group_dict_list:
                        self.group_dict_list.get(g).UpdateSkillSet({name})
                        #print(name,"added in",g,".")
                    else:
                        self.group_dict_list[g] = Group(g,{name})
                        #print("new group:",g,"have been created and added",name,".")
        else:
            self.UpdateSkillDictList(name,groups,prerequisites)
                    
    def UpdateSkillDictList(self,name,groups,prerequisites):
        if name in self.skill_dict_list:
            self.skill_dict_list[name].UpdateGroupSet(groups)
            if prerequisites is not None:
                self.skill_dict_list[name].UpdatePrerequisites(prerequisites)
                #print(name,"prerequisites updated.")

    def AddGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            self.UpdateGroupDictList(name,skills)
        else:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                    self.skill_dict_list[s].UpdateGroupSet({name})
                    found_set.add(s)  
                    #print(s,"added in",name,"group set.")
            self.group_dict_list[name] = Group(name,found_set)

    def UpdateGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                      found_set.add(s)  
            self.group_dict_list[name].UpdateSkillSet(found_set)
        else:
            self.AddGroupDictList(name,skills)

    def AddAliasList(self,key,value):
        if key not in self.alias_dict_list:
            self.alias_dict_list[key]=value
            #print(key,"alias as",value,"created.")
            
    def AddIgnoreList(self,name):
        self.ignore_set.add(name)

    def AddRoleDictList(self,name,skills,groups):
        if name in self.role_dict_list:
            if skills is not None:
                skill_found_set = set()
                for s in skills:
                    if s in self.skill_dict_list:
                        skill_found_set.add(s)  
                self.role_dict_list[name].UpdateSkillSet(skill_found_set)
            if groups is not None:
                group_found_set = set()
                for g in groups:
                    if g in self.group_dict_list:
                        group_found_set.add(g)  
                self.role_dict_list[name].UpdateGroupSet(group_found_set)
        else:
            self.role_dict_list[name] = Role(name,skills,groups)

    def ExportSkillDictList(self):
        file_path = "skills.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Groups","Prerequisites"])
            for key, value in self.skill_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                groups =""
            
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                pre = ""
                for p in value.prerequisite:
                    pre += "["
                    pre += p
                    pre +="]"
                writer.writerow([name,search,path,groups,pre])
            file.close()

    def ExportGroupDictList(self):
        file_path = "groups.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","skills"])
            for key, value in self.group_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                writer.writerow([name,search,path,skills])
            file.close()

    def LearningDocumentPrepare(self,company,skills):
        
        
        document = ""
        #new_list = [x for x in job_skills if x not in resume_skills]

        #check leetcode required
    

        # get all the learning resource reference link
        for s in skills:
            key = s
            if s in self.alias_dict_list:
                key = alias_dict_list.get(s)
                
            if key in self.skill_dict_list:
                # do something
                continue
            elif key in self.group_dict_list:
                # do something
                continue
            elif key in self.role_dict_list:
                # do something
                continue
            elif key in self.ignore_list:
                continue
            else:
                self.not_found_set.add(key)
        
         




In [17]:
test = TechStack()
# top programming list
name_list = []
name_list.append("javascript")
name_list.append("html")
name_list.append("css")
name_list.append("python")
name_list.append("sql")
name_list.append("bash")
name_list.append("java")
name_list.append("c#")
name_list.append("c++")
name_list.append("c")
name_list.append("php")
name_list.append("powershell")
name_list.append("go")
name_list.append("rust")
name_list.append("kotlin")
name_list.append("ruby")
name_list.append("lua")
name_list.append("dart")
name_list.append("assembly")
name_list.append("swift")
name_list.append("r")
name_list.append("visual basic .net")
name_list.append("matlab")
name_list.append("vba")
name_list.append("groovy")
name_list.append("delphi")
name_list.append("scala")
name_list.append("perl")
name_list.append("elixir")
name_list.append("objective-c")
name_list.append("haskell")
name_list.append("gdscript")
name_list.append("lisp")
name_list.append("solidity")
name_list.append("clojure")
name_list.append("julia")
name_list.append("erlang")
name_list.append("f#")
name_list.append("fortran")
name_list.append("prolog")
name_list.append("zig")
name_list.append("ada")
name_list.append("ocaml")
name_list.append("sas")
name_list.append("crystal")
name_list.append("apl")
name_list.append("flow")
name_list.append("raku")
name_list.append("racket")
name_list.append("xhtml")
for i in range(len(name_list)):
    keyword = name_list[i] + " programming"
    test.AddSkillDictList(name_list[i], "code", keyword,{"programming"})
    

skill_set ={"c++","java","python","c","c#","javascript","typescript","php","swift","kotlin","dart","go","ruby","scala","rust","racket","erlang","elixir"}
test.AddGroupDictList("leetcode",skill_set)

# top framework/libraey
name_list.clear()
name_list.append(".net")
name_list.append("numpy")
name_list.append("pandas")
name_list.append(".net framework")
name_list.append("spring")
name_list.append("rabbitmq")
name_list.append("tensorflow")
name_list.append("scikit-learn")
name_list.append("flutter")
name_list.append("kafka")
name_list.append("torch")
name_list.append("react native")
name_list.append("opencv")
name_list.append("electron")
name_list.append("opengl")
name_list.append("qt")
name_list.append("cuda")
name_list.append("keras")
name_list.append("spark")
name_list.append("swiftui")
name_list.append("xamarin")
name_list.append("lonic")
name_list.append("transformers")
name_list.append("gtk")
name_list.append("cordova")
name_list.append(".net maui")
name_list.append("hadoop")
name_list.append("tauri")
name_list.append("capacitor")
name_list.append("tidyverse")
name_list.append("quarkus")
name_list.append("ktor")
name_list.append("mfc")
name_list.append("jax")
name_list.append("micronaut")
name_list.append("uno platform")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "code", keyword)

#top frontend
name_list.clear()
name_list.append("react")
name_list.append("tailwind css")
name_list.append("angular")
name_list.append("vue.js")
name_list.append("bootstrap")
name_list.append("mdl")
name_list.append("bulma")
name_list.append("mdl")
name_list.append("semantic ui")
name_list.append("foundation")
name_list.append("ulkit")
for i in range(len(name_list)):
    keyword = name_list[i] + " frontend"
    test.AddSkillDictList(name_list[i], "code", keyword, {"frontend"})

# top backend
name = "spring boot"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"java"})
name = "django"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"python"})
name = "express.js"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"node.js"})
name = "laravel"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"php"})
name = "ruby on rails"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"ruby"})
name = "asp.net core"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"c#"})
name = "nestjs"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"typescript","node.js"})
name = "koa.js"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"node.js"})
name = "phoenix"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"elixir"})

# top web framework
name_list.clear()
name_list.append("node.js")
name_list.append("react")
name_list.append("jquery")
name_list.append("express")
name_list.append("angular")
name_list.append("next.js")
name_list.append("asp.net.core")
name_list.append("vue.js")
name_list.append("wordpress")
name_list.append("asp.net")
name_list.append("flask")
name_list.append("spring boot")
name_list.append("django")
name_list.append("laravel")
name_list.append("fastapi")
name_list.append("angularjs")
name_list.append("svelte")
name_list.append("ruby on rails")
name_list.append("nestjs")
name_list.append("blazor")
name_list.append("nuxt.js")
name_list.append("symfony")
name_list.append("deno")
name_list.append("gatsby")
name_list.append("fastify")
name_list.append("phoenix")
name_list.append("drupal")
name_list.append("codelgniter")
name_list.append("solid.js")
name_list.append("remix")
name_list.append("elm")
name_list.append("play")
test.AddAliasList("play framework","play")
name_list.append("lit")
name_list.append("qwik")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "code", keyword, {"web development"})

# top mobile development framework
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("swiftic")
name_list.append("kotlin")
name_list.append("ionic")
name_list.append("xamarin")
name_list.append("corona")
name_list.append("titanium")
name_list.append("nativescript")
name_list.append("jquery mobile")
for i in range(len(name_list)):
    keyword = name_list[i] + " mobile development framework"
    test.AddSkillDictList(name_list[i], "code", keyword, {"mobile development"})

# top microservices
name = "spring boot"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "spring cloud"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "express.js"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"node.js"})
name = "django"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "django rest"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "go kit"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"go"})
name = "micronaut"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java","groovy","kotlin"})
name = "fastapi"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "helidon"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "lagom"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java","scala"})
name = "quarkus"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})

# top test automation frameworks
name = "selenium"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","python","c#"})
name = "appium"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing","mobile development"})
name = "testng"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","junit"})
name = "junit"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java"})
name = "cucumber"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
name = "testcomplete"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"javascript","python","visual basic"})
name = "robot"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
name = "cypress"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"javascript"})
name = "junit jupiter"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","junit"})
name = "pytest"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"python"})
name = "rspec"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
name = "galen"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing","web development"})

# top distributed
name = "hadoop"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "mapreduce"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "spark"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "hive"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache hive","hive")
name = "storm"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "samza"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "flink"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "heron"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "kudu"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "presto"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "zookeeper"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "centos"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
# mq
name = "rabbitmq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "ironmq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "ibm mq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "mulesoft"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "pubsub+"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "kafka"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "zeromq"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "tibco"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "activemq"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})

# top ai framework
name = "torch"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "scikit-learn"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
name = "tensorflow"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "keras"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
name = "langchain"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning"})
name = "transformers"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning"})
name = "opennn"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "openai"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning"})
name = "pybrain"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "ibm watson"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "cntk"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "dl4j"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "theano"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "mxnet"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "caffe"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence"})
name = "xgboost"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","machine learning"})
name = "opencv"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "viso suite"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "yolo"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "boofcv"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "openvino"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "simplecv"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "deepface"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","computer vision"})
name = "spacy"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","natural language processing" })
name = "xlm"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","natural language processing" })
name = "chainer"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","natural language processing" })
name = "gensim"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","natural language processing" })
name = "nltk"
test.AddSkillDictList(name, "code", name + " framework", {"artificial intelligence","natural language processing" })

# web crawler
name = "selenium"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"})
name = "selenium base"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"selenium"})
name = "scrapy"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})
name = "mechanicalsoup"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})
name = "beautiful soup"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})

#database
name = "oracle database"
test.AddSkillDictList(name, "database", name, {"database"})
name = "mysql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "microsoft sql"
test.AddSkillDictList(name, "database", name + " database", {"database","warehouse"})
name = "postgresql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "mongodb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","document database"})
name = "cassandra"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","distributed"})
name = "redis"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","in memory database"})
name = "neo4j"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","graph database"})
name = "sqlite"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "mariadb"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "elasticsearch"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "ibm db2"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "couchbase"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "orientdb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql"})
name = "firebirdsql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "influxdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "kdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "prometheus"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "chroma"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "pinecone"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "faiss"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "tibero"
test.AddSkillDictList(name, "database", name + " database", {"database"})

#container
name = "docker"
test.AddSkillDictList(name, "container", name + " container",{"container","devops"} )
name = "lxc"
test.AddSkillDictList(name, "container", name + " container",{"container","devops"} )
name = "container linux"
test.AddSkillDictList(name, "container", name ,{"container","devops"} )
name = "portainer"
test.AddSkillDictList(name, "container", name + " container",{"container","devops"} )
name = "messos"
test.AddSkillDictList(name, "container", name + " container",{"container","devops"} )
name = "kubernetes"
test.AddSkillDictList(name, "container", name + " container",{"container","devops"} )

# cl/cd
name = "spacelift"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "github"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "jenkins"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "buddy"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "teamcity"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "circleci"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "tracisci"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "gitlab"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "bitbucket"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "harness"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "semaphore"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )
name = "git"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd","devops"} )

# monitoring tool
name = "datadog"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "grafana"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "nagios"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "micro focus sitescope"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "sensu"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "prometheus"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "observium"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "checkmk"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "new relic"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "dynatrace"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "atera"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "paessler prtg"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "zabbix"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "appdynamics"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "logrocket"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "logz.io"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )

#aws
name = "s3"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","storage"})
name = "efs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "fsx"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "ebs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "amazon file cache"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "aws drs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "neptune"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","graph database"})
name = "rds"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","relational"})
name = "dynamodb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","nosql"})
name = "redshift"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","warehouse"})
name = "aws database migration service"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "elasticache"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","data store"})
name = "memorydb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","in memory database"})
name = "documentdb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","document databaset"})
name = "timestream"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","database","time series"})
name = "aws glue"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "athena"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","big data"})
name = "emr"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","big data"})
name = "amazon sns"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","mq","big data"})
name = "lake formation"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "sagemaker"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","ai"})
name = "ecs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","container"})
name = "fargate"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","container"})
name = "lambda"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws"})
name = "cloudwatch"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","monitoring"})
name = "sqs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","mq"})
name = "codepipeline"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud services","aws","ci cd"})

#azure
name = "cosmosdb"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud services","azure","database","nosql"})
name = "azure sql database"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure","database","relational"})
name = "azure blobs"
test.AddSkillDictList(name, "azure", name, {"cloud services","azure","storage"})
name = "azure file"
test.AddSkillDictList(name, "azure", name, {"cloud services","azure"})
name = "azure elastic san"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure"})
name = "azure tables"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure"})
name = "azure queues"
test.AddSkillDictList(name, "azure", name, {"cloud services","azure"})
name = "azure managed disks"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure"})
name = "azure container storage"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure"})
name = "azure container"
test.AddSkillDictList(name, "azure", name, {"cloud services","azure","container"})
name = "azure data lake"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure"})
name = "service bus"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud services","azure","mq"})
name = "hdinsight"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud services","azure","mq"})
name = "azure synapse analytics"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure","warehouse","big data"})
name = "azure ai"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure","ai"})
name = "azure montior"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure","monitoring"})
name = "azure pipelines"
test.AddSkillDictList(name, "azure",  name, {"cloud services","azure","ci cd"})
name = "arm"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud services","azure"})

#google cloud
name = "cloud sql"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","relational"})
name = "alloydb"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","relational"})
name = "spanner"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","relational"})
name = "bigtable"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","key value"})
name = "firestore"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","document database"})
name = "firebase realtime database"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","document database"})
name = "memorystore"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","in memory database"})
name = "mongodb atlas"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","database","nosql"})
name = "bigquery"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","warehouse","big data"})
name = "dataproc"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud"})
name = "google cloud storage"
test.AddSkillDictList(name, "google cloud", name, {"cloud services","google cloud","storage"})
name = "vertex ai"
test.AddSkillDictList(name, "google cloud", name, {"cloud services","google cloud","ai"})
name = "cloud tasks"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","mq"})
name = "pub sub"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","mq"})
name = "gke"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud services","google cloud","container"})
name = "google cloud monitoring"
test.AddSkillDictList(name, "google cloud", name, {"cloud services","google cloud","monitoring"})
name = "cloud buid"
test.AddSkillDictList(name, "google cloud", "google "+ name, {"cloud services","google cloud","ci cd"})

name = "tomcat"
test.AddSkillDictList(name, "application servers", name + " application servers",{"application servers"} )
name = "glassfish"
test.AddSkillDictList(name, "application servers", name + " application servers",{"application servers"} )
name = "jetty"
test.AddSkillDictList(name, "application servers", name + " application servers",{"application servers"} )
name = "wildfly"
test.AddSkillDictList(name, "application servers", name + " application servers",{"application servers"} )

# pen testing
name = "nmap"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "w3af"
test.AddSkillDictList(name, "security", name +" pen test" , {"security"})
name = "tenable nessus"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "onsecurity"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "wireshark"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "zap"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "nikto"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "cobalt"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "rapid7"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})
name = "astra security"
test.AddSkillDictList(name, "security", name + " pen test", {"security"})

# top blockchain
name = "ethereum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ethereum 2.0"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ibm blockchain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hyperledger"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "eos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "corda"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "ripple"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "quorum"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "iota"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "neo"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tezos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "stellar"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "sui network"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "hedera"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "aptos"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "flow"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "fantom"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "solana"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "tron"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "binance smart chain"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
name = "icp"
test.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})

# top game developer
name = "unreal engine"
test.AddSkillDictList(name, "game", name + " engine", {"game development"},{"c++"})
name = "unity"
test.AddSkillDictList(name, "game", name + " engine", {"game development"},{"c#"})
name = "godot"
test.AddSkillDictList(name, "game", name + " engine", {"game development"},{"c#"})
name = "cocos2d"
test.AddSkillDictList(name, "game", name + " engine", {"game development"})
name = "rpg maker"
test.AddSkillDictList(name, "game", name + " engine", {"game development"})
name = "vulkan"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game development"})
name = "opengl"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game development"})
name = "directx"
test.AddSkillDictList(name, "game", name + " graphics framework", {"game development"})
name = "physx"
test.AddSkillDictList(name, "game", name + " physics engine", {"game development"})
name = "bullet"
test.AddSkillDictList(name, "game", name + " physics engine", {"game development"})
name = "open dynamics engine"
test.AddSkillDictList(name, "game", name + " physics engine", {"game development"})
name = "fmod"
test.AddSkillDictList(name, "game", name + " audio framework", {"game development"})
name = "glm"
test.AddSkillDictList(name, "game", name + " math framework", {"game development"})
name = "assimp"
test.AddSkillDictList(name, "game", name + " library", {"game development"})

#quantum
name = "tensorflow quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "microsoft quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "qiskit"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "cuquantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})

name = "windows"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "mac"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "linux"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "ios"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "android"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )
name = "solaris"
test.AddSkillDictList(name, "os", name + " technology", {"os"} )

name = "cisco ios"
test.AddSkillDictList(name, "network", name ,{"network","security"} )
name = "san"
test.AddSkillDictList(name, "network", name + " network", {"network","storage"} )
name = "dns"
test.AddSkillDictList(name, "network", name + " network", {"network"} )
name = "routing"
test.AddSkillDictList(name, "network", name + " network", {"network"} )
name = "bridge"
test.AddSkillDictList(name, "network", name + " network", {"network"} )
name = "vpn"
test.AddSkillDictList(name, "network", name + " network", {"network"} )
name = "ip"
test.AddSkillDictList(name, "network", name + " network", {"network"} )
name = "virtual servers"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "wlan"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "switches"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "network troubleshooting"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "iis"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "tcpdump"
test.AddSkillDictList(name, "network", name , {"network"} )
name = "web services"
test.AddSkillDictList(name, "other", name )
name = "suse"
test.AddSkillDictList(name, "cloud services", name + " cloud",{"cloud services"} )
name = "snowflake"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "databricks"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "citrix"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "salesforce"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "vmware vsphere"
test.AddSkillDictList(name, "cloud services", name, {"cloud services","vmware"} )
name = "vmware vcenter"
test.AddSkillDictList(name, "cloud services", name, {"cloud services","vmware"} )
name = "servicenow"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "akamai"
test.AddSkillDictList(name, "cloud services", name, {"cloud services"} )
name = "terraform"
test.AddSkillDictList(name, "code", name )
name= "oracle adf"
test.AddSkillDictList(name, "code", name + " framework"  )
name= "aspnet mvc"
test.AddSkillDictList(name, "code", name + " framework"  )
name= "pyspark"
test.AddSkillDictList(name, "code", name + " framework", None,{"python", "spark"}  )



name= "data structures"
test.AddSkillDictList(name, "other", name + " programming terminology" ,{"programming terminology"})
name= "factory"
test.AddSkillDictList(name, "other", name + " programming terminology" ,{"programming terminology"})
name= "multithreading"
test.AddSkillDictList(name, "other", name + " programming terminology" ,{"programming terminology"})
name= "debugging"
test.AddSkillDictList(name, "other", name + " programming terminology" ,{"programming terminology"})

name= "business process reengineering"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "sdlc"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "alm"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "agile"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "software development"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "application development"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "disaster recovery"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "maintenance"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})
name= "replication"
test.AddSkillDictList(name, "other", name + " technology term" ,{"technology term"})

name ="jira"
test.AddSkillDictList(name, "other", name + " technology", {"documentation"} )
name ="ms project"
test.AddSkillDictList(name, "other", name + " technology", {"documentation"} )

name = "postman"
test.AddSkillDictList(name, "other", name,{"monitoring","automation testing","web development"} )
name = "lims"
test.AddSkillDictList(name, "other", name + " technology" )
name = "interpreting"
test.AddSkillDictList(name, "other", name + " technology" )
name = "zendesk"
test.AddSkillDictList(name, "other", name + " technology" )
name = "arcgis"
test.AddSkillDictList(name, "other", name + " technology"  )
name= "solidworks"
test.AddSkillDictList(name, "other", name + " technology"  )
name= "oracle erp"
test.AddSkillDictList(name, "other", name + " technology"  )
name= "account management"
test.AddSkillDictList(name, "other", name + " technology"  )
name= "vlookup"
test.AddSkillDictList(name, "other", name + " technology"  )
name= "mcsd"
test.AddSkillDictList(name, "other", name + " technology" ) 
name= "ccnp"
test.AddSkillDictList(name, "other", name + " technology" )
name= "vcp"
test.AddSkillDictList(name, "other", name + " technology" )
name= "sccm"
test.AddSkillDictList(name, "other", name + " technology" )
name= "j2ee"
test.AddSkillDictList(name, "other", name + " technology" )
name= "cadence"
test.AddSkillDictList(name, "other", name + " technology",{"ai"} )
name= "uml"
test.AddSkillDictList(name, "other", name + " technology" )
name= "cissp"
test.AddSkillDictList(name, "other", name + " technology" )
name= "maven"
test.AddSkillDictList(name, "other", name + " technology", None, {"java"} )
name= "sap"
test.AddSkillDictList(name, "other", name + " technology" )

test.ExportSkillDictList()
test.ExportGroupDictList()

In [18]:
skill_set ={"c++","java","python","c","c#","javascript","typescript","sql"}
test.AddRoleDictList("architecture",skill_set,{"database","cloud services"})
test.AddRoleDictList("system administrator",{"account management"},{"monitoring"})
test.AddRoleDictList("reliability engineering",{"disaster recovery","replication","maintenance"},{"monitoring","cloud services","database","ci cd"})
test.AddRoleDictList("cybersecurity",None,{"network", "security","cloud services"})
test.AddRoleDictList("data analytics",None,None)

In [20]:
test.AddAliasList("technical architecture","architecture")
test.AddAliasList("solution architecture","architecture")
test.AddAliasList("application architecture","architecture")
test.AddAliasList("architect","architecture")
test.AddAliasList("reliability","reliability engineering")
test.AddAliasList("microsoft azure","azure")
test.AddAliasList("html5","html")
test.AddAliasList("google cloud platform","google cloud")
test.AddAliasList("vertex","vertex ai")
test.AddAliasList("windows 10","windows")
test.AddAliasList("amazon aws","aws")
test.AddAliasList("test cases","automation testing")
test.AddAliasList("xp","windows")
test.AddAliasList("cryptography","blockchain")
test.AddAliasList("gcp","google cloud")
test.AddAliasList("network security","cybersecurity")
test.AddAliasList("security audits","cybersecurity")
test.AddAliasList("website development","web development")
test.AddAliasList("web security","cybersecurity")
test.AddAliasList("continuous integration","ci cd")
test.AddAliasList("weblogic","web development")
test.AddAliasList("intelligent systems","ai")
test.AddAliasList("iphone","ios")
test.AddAliasList("unit testing","automation testing")
test.AddAliasList("test planning","automation testing")
test.AddAliasList("test automation","automation testing")
test.AddAliasList("uat","automation testing")
test.AddAliasList("administration","system administrator")
test.AddAliasList("enterprise architecture","architecture")
test.AddAliasList("technical documentation","documentation")
test.AddAliasList("window 7","windows")
test.AddAliasList("website maintenance","maintenance")
test.AddAliasList("relational databases","relational")
test.AddAliasList("agile software development","agile")                  
test.AddAliasList("agile methodology","agile")       
test.AddAliasList("cloud-based","cloud services")
test.AddAliasList("problem solving","leetcode")
test.AddAliasList("scripting","programming")
test.AddAliasList("snowflake computing","snowflake")
test.AddAliasList("Snowflake cloud data warehouse","snowflake")
test.AddAliasList("debugging of applications","debugging")
test.AddAliasList("pcf","cloud services")
test.AddAliasList("routers","routing")
test.AddAliasList("sun solaris 7","solaris")
test.AddAliasList("solaris (operating system)","solaris")
test.AddAliasList("arcgis server","arcgis")
test.AddAliasList("sap basis","sap")
test.AddAliasList("information security management","cybersecurity")
test.AddAliasList("vmware infrastructure","vmware")
test.AddAliasList("hdfs","hadoop")
test.AddAliasList("version control","ci cd")    
test.AddAliasList("public cloud","cloud services")    

In [21]:
test.AddIgnoreList("entertainment")
test.AddIgnoreList("mastering")
test.AddIgnoreList("quality management")
test.AddIgnoreList("emergency response")
test.AddIgnoreList("ips")
test.AddIgnoreList("telemarketing")
test.AddIgnoreList("customer advocacy")
test.AddIgnoreList("product development")
test.AddIgnoreList("cost management")
test.AddIgnoreList("online travel")
test.AddIgnoreList("service delivery")
test.AddIgnoreList("direct sales")
test.AddIgnoreList("formulation")
test.AddIgnoreList("project risk management")
test.AddIgnoreList("television")
test.AddIgnoreList("mergers")
test.AddIgnoreList("critical thinking")
test.AddIgnoreList("social services")
test.AddIgnoreList("customer negotiations")
test.AddIgnoreList("general insurance")
test.AddIgnoreList("managing absence")
test.AddIgnoreList("uc")
test.AddIgnoreList("team spirit")
test.AddIgnoreList("public speaking")
test.AddIgnoreList("field work")
test.AddIgnoreList("selling")
test.AddIgnoreList("underwriting")
test.AddIgnoreList("office administration")
test.AddIgnoreList("business requirements")
test.AddIgnoreList("housekeeping")
test.AddIgnoreList("financial services")
test.AddIgnoreList("banking")
test.AddIgnoreList("writing skills")
test.AddIgnoreList("indonesian language")
test.AddIgnoreList("b2b")
test.AddIgnoreList("people management experience")
test.AddIgnoreList("electronics")
test.AddIgnoreList("process improvement")
test.AddIgnoreList("grants")
test.AddIgnoreList("grants")
test.AddIgnoreList("graphic design")
test.AddIgnoreList("research design")
test.AddIgnoreList("security research")
test.AddIgnoreList("ux")
test.AddIgnoreList("usability testing")
test.AddIgnoreList("user research")
test.AddIgnoreList("thought leadership")
test.AddIgnoreList("technical support")
test.AddIgnoreList("technology solutions")
test.AddIgnoreList("ict strategy")
test.AddIgnoreList("project budget")
test.AddIgnoreList("photoshop")
test.AddIgnoreList("ability to work independently")
test.AddIgnoreList("stakeholder engagement")
test.AddIgnoreList("help desk support")
test.AddIgnoreList("team development")
test.AddIgnoreList("ict policy")
test.AddIgnoreList("trading")
test.AddIgnoreList("bloomberg")
test.AddIgnoreList("user experience design")
test.AddIgnoreList("assurance")
test.AddIgnoreList("business services")
test.AddIgnoreList("ms word")
test.AddIgnoreList("requirements analysis")
test.AddIgnoreList("user interface and eser experience optimisation")
test.AddIgnoreList("compsites")
test.AddIgnoreList("supplier management")
test.AddIgnoreList("orchestration")
test.AddIgnoreList("management skills")
test.AddIgnoreList("outlook")
test.AddIgnoreList("operational excellence")
test.AddIgnoreList("estimates")
test.AddIgnoreList("data governance")
test.AddIgnoreList("compliance")
test.AddIgnoreList("presales")
test.AddIgnoreList("benchmarking")
test.AddIgnoreList("benchmarking")
test.AddIgnoreList("organizational design")
test.AddIgnoreList("basic it skills")
test.AddIgnoreList("business stakeholder management")
test.AddIgnoreList("manual testing")
test.AddIgnoreList("contract management")
test.AddIgnoreList("swimming")
test.AddIgnoreList("information management")
test.AddIgnoreList("information technology and services industry")
test.AddIgnoreList("protocol")
test.AddIgnoreList("humility")
test.AddIgnoreList("wireframes")
test.AddIgnoreList("pmp")
test.AddIgnoreList("manage change")
test.AddIgnoreList("technical direction")
test.AddIgnoreList("laptops")
test.AddIgnoreList("business process")
test.AddIgnoreList("orc")
test.AddIgnoreList("software design")
test.AddIgnoreList("information technology audit")
test.AddIgnoreList("technology management")
test.AddIgnoreList("customer relationship management crm")
test.AddIgnoreList("project timeline")
test.AddIgnoreList("audit")
test.AddIgnoreList("escalation management")
test.AddIgnoreList("turnkey projects")
test.AddIgnoreList("visual merchandising")
test.AddIgnoreList("strategic business development management")
test.AddIgnoreList("thai language")
test.AddIgnoreList("actuarial consulting")
test.AddIgnoreList("attention to details")
test.AddIgnoreList("vietnamese language")
test.AddIgnoreList("localization")
test.AddIgnoreList("microsoft office")
test.AddIgnoreList("derivatives")
test.AddIgnoreList("pricing")
test.AddIgnoreList("presentation skills")
test.AddIgnoreList("inventory management")
test.AddIgnoreList("screening")
test.AddIgnoreList("fabrication")
test.AddIgnoreList("advocacy")
test.AddIgnoreList("physical security")
test.AddIgnoreList("business strategy")
test.AddIgnoreList("responding to rfp")
test.AddIgnoreList("need analysis")
test.AddIgnoreList("illustrator")
test.AddIgnoreList("technical sales")
test.AddIgnoreList("hunting")
test.AddIgnoreList("sharepoint")
test.AddIgnoreList("lifelong learning")
test.AddIgnoreList("telephony")
test.AddIgnoreList("communication skills")
test.AddIgnoreList("vendor relationships")
test.AddIgnoreList("regulatory compliance")
test.AddIgnoreList("scheduling")
test.AddIgnoreList("personal development")
test.AddIgnoreList("change control")
test.AddIgnoreList("tuning")
test.AddIgnoreList("workflow design")
test.AddIgnoreList("team worker")
test.AddIgnoreList("actuaries")
test.AddIgnoreList("data centre")
test.AddIgnoreList("teamoriented")
test.AddIgnoreList("business acumen")
test.AddIgnoreList("operations management")
test.AddIgnoreList("multitasking skills")
test.AddIgnoreList("visio")
test.AddIgnoreList("pressure")
test.AddIgnoreList("lead generation")
test.AddIgnoreList("airwatch")
test.AddIgnoreList("vios")
test.AddIgnoreList("exceptional customer service")
test.AddIgnoreList("human factors")
test.AddIgnoreList("web design")
test.AddIgnoreList("technical communication")
test.AddIgnoreList("articulate")
test.AddIgnoreList("accounting")

In [ ]:
# stop at 296